## **1. Introduction**
### **1.1 Contexte du Projet**
- Ce projet est un jeu d'exploration de donjons au tour par tour développé avec Pygame.
- Les joueurs contrôlent des héros avec des compétences uniques pour combattre des monstres.
- Les principales fonctionnalités incluent la gestion de la carte, le déplacement des unités, les animations d'attaques et l'interaction des attributs.

### **1.2 Fonctionnalités principales**
- **Gestion des donjons** : Génération dynamique et rendu des cartes.
- **Gestion des unités** : Déplacement, mise à jour des états et utilisation de compétences.
- **Effets d'attaques** : Animations avancées avec des effets basés sur les attributs.
- **Support multijoueur** : Système de tours pour alterner entre plusieurs joueurs.


## **2. Structure du Projet**
Les principaux modules et leurs fonctions sont les suivants :

- **`main.py`** : La boucle principale du jeu et la logique des tours.
  - Gère l'alternance des tours et les interactions des joueurs.
  - Coordonne les actions des sous-modules pour les animations et la mise à jour des cartes.

- **`HerosGenerator.py`** : Définit les héros et leurs compétences uniques.
  - Chaque héros a des attaques et des capacités spécifiques, comme "Water Splash" ou "Flame Burst".

- **`spriteManager.py`** : Gestion des héros et des monstres, y compris les états et les attaques.
  - Met à jour les états tels que "Gel", "Ralentissement" et "Brûlure".
  - Applique les dégâts en fonction des attributs des unités.

- **`monsters.py`** : Définit les monstres et leur comportement.
  - Contient des monstres avec des attaques et des capacités distinctes.
  - Permet aux monstres de se déplacer de manière aléatoire sur la carte.

- **`dungeonManager.py`** : Gestion et rendu des cartes de donjon.
  - Génère des tuiles de terrain avec des niveaux d'élévation.
  - Met en évidence les zones de déplacement et les cibles possibles pour les attaques.

- **`mediaManager.py`** : Gestionnaire de ressources pour charger les images et les sons.
  - Charge les sprites des personnages, les effets visuels et les fichiers audio.

- **`Type_Effectiveness.py`** : Logique des interactions des attributs.
  - Détermine les effets comme les dégâts supplémentaires, la guérison ou les statuts appliqués.

- **`utils.py`** : Fournit des fonctions utilitaires comme la mise à jour des cartes.
  - Gère les modifications des cartes en fonction des actions des unités.

- **`windowManager.py`** : Gestion de la fenêtre et de l'interface utilisateur.
  - Centre les éléments sur l'écran et gère les dimensions.

- **`anim.py`** : Gestion des animations des attaques.
  - Joue des animations spécifiques pour les attaques comme "Thunder Strike" ou "Fireball".


## **3. Stack Technologique**
- **Langage** : Python
- **Bibliothèque** : Pygame, os, sys, random, math
- **Outils de développement** : Jupyter Notebook, IDE

## **4. Conception et Implémentation des Modules**
### **4.1 Module Principal**
- **Description** : Gère la boucle principale du jeu, y compris le passage des tours, le déplacement et les attaques.
- **Fonctionnalités principales** :
  - Implémentation de la logique des tours.
  - Réponse aux entrées utilisateur pour les déplacements et les actions.
  - Coordination avec les sous-modules pour gérer les animations et mettre à jour les cartes.

In [ ]:
    def run_game_loop(self):
        """Runs the main game loop."""
        # Start with Dracko's turn
        players = [self.dracko_player, self.second_player]
        active_player_index = 0
        players[active_player_index].set_active(True)

        attack_animation_playing = False
        attack_animation_position = None
        attack_animation_type = None

        self.fireball_group = pygame.sprite.Group()
        self.tsunami_group = pygame.sprite.Group()
        self.flame_burst_group = pygame.sprite.Group()
        self.ice_spike_group = pygame.sprite.Group()
        self.blizzard_group = pygame.sprite.Group()
        self.light_storm_group = pygame.sprite.Group()
        current_unit_index = 0
        last_turn_switch_time = 0 
        switch_cooldown = 700
        

        highlighted_positions = set()
        new_zone_position = None
        while True:
            pygame.event.pump()  # updating the events queue from the os
            key_input = pygame.key.get_pressed()

            current_time = pygame.time.get_ticks()
            # print(current_time)


            # Handle zone things
            score_to_add_new_zone = 200
            if players[0].score >= score_to_add_new_zone or players[1].score >= score_to_add_new_zone:
                self.add_zone = True
                new_zone_position = (11, 4)
            self.handle_zone(players, new_zone_position)




            # Handle unit switching within the active player
            if key_input[K_TAB]:
                current_unit_index = (current_unit_index + 1) % len(players[active_player_index].sprite_managers)

                self.poll_events_with_timeout(20)

            # Handle turn switching with cooldown
            if (
                (players[active_player_index].is_turn
                and players[active_player_index].played == 1
                and current_time - last_turn_switch_time > switch_cooldown)
                or (players[active_player_index].played == 1)
            ):
                players[active_player_index].played = 0
                players[active_player_index].set_active(False)

                """-----------------------------"""
                for sprite in players[active_player_index].sprite_managers:
                    sprite.update_status_effects()

                active_player_index = (active_player_index + 1) % len(players)
                players[active_player_index].set_active(True)
                last_turn_switch_time = current_time  # Update the timestamp

            if current_unit_index >= len(players[active_player_index].sprite_managers):
                current_unit_index = 0
            active_unit = players[active_player_index].sprite_managers[current_unit_index]
            unit_position = active_unit.mapPosition
            # unit_position = players[active_player_index].sprite_managers[current_unit_index].mapPosition

            # Process movement only for the active player
            # Flag to store paralysis check result for the current turn
            if not hasattr(active_unit, "paralyze_checked"):
                active_unit.paralyze_checked = False
                active_unit.Trigger_paralysis = False

            if active_unit.is_paralyze and not active_unit.paralyze_checked:
                active_unit.paralyze_checked = True
                if random.random() < 0.5:  # 50% chance
                    active_unit.Trigger_paralysis = True
                    print(f"{active_unit.name} is paralyzed and cannot take actions this turn.")
                else:
                    active_unit.Trigger_paralysis = False
                    print(f"{active_unit.name} resists paralysis and can act this turn!")
            self.menu_open = active_unit.menu_open # mode menu ou pas recuperer des que je clique sur m ca devient True
            # print(f"menu{active_unit.menu_open}")
            if not self.menu_open:
                if players[active_player_index].is_turn() :

                    if not active_unit.Trigger_paralysis:
                        if key_input[K_UP]:
                            if active_unit.is_frozen :
                                print(f"{active_unit.name} is frozen and cannot take actions this turn.")
                            else:
                                players[active_player_index].take_turn(1, current_unit_index, highlighted_positions, active_unit.mapPosition)
                        elif key_input[K_DOWN]:
                            if active_unit.is_frozen :
                                print(f"{active_unit.name} is frozen and cannot take actions this turn.")
                            else :
                                players[active_player_index].take_turn(2, current_unit_index, highlighted_positions, active_unit.mapPosition)
                        elif key_input[K_LEFT]:
                            if active_unit.is_frozen :
                                print(f"{active_unit.name} is frozen and cannot take actions this turn.")
                            else:
                                players[active_player_index].take_turn(3, current_unit_index, highlighted_positions, active_unit.mapPosition)
                        elif key_input[K_RIGHT]:
                            if active_unit.is_frozen :
                                print(f"{active_unit.name} is frozen and cannot take actions this turn.")
                            else:
                                players[active_player_index].take_turn(4, current_unit_index, highlighted_positions, active_unit.mapPosition)
                        elif key_input[K_SPACE]:
                            if active_unit.is_frozen :
                                print(f"{active_unit.name} is frozen and cannot take actions this turn.")
                            else :
                                players[active_player_index].take_turn(0, current_unit_index, highlighted_positions, active_unit.mapPosition)
                            for enemy_sprite in players[not (active_player_index)].sprite_managers:
                                enemy_sprite.defense = False
                """else:
                    print(f"{active_unit.name} is frozen and cannot take actions this turn.")"""
                self.target_position_sprite.mapPosition = [active_unit.mapPosition[0], active_unit.mapPosition[1]]

            # Update the game screen
            self.screen.blit(self.background, (0, 0))
            #for monster in self.monsters:
                #monster.draw(self.screen)

            if active_unit.attack_selected: # gerer une seule tuile apres avoir choisi une attaque
                if key_input[K_UP]:
                    self.target_position_sprite.update(1)
                elif key_input[K_DOWN]:
                    self.target_position_sprite.update(2)
                elif key_input[K_LEFT]:
                    self.target_position_sprite.update(3)
                elif key_input[K_RIGHT]:
                    self.target_position_sprite.update(4)
                unit_position = self.target_position_sprite.mapPosition

            attack_position = active_unit.handle_attacks(key_input, self.screen, unit_position)  # ou est ce que tu as appuyé sur entrée quand tu geres une suile pour lattaque
            # print(active_unit.selected_attack)
            if active_unit.defense:
                attack_position = None
                # change the player once defense
                players[active_player_index].played = 0
                players[active_player_index].set_active(False)
                active_player_index = (active_player_index + 1) % len(players)
                players[active_player_index].set_active(True)
                last_turn_switch_time = current_time
                #print(f"{active_unit.name} is defense and cannot take actions this turn.")
            selected_attack = active_unit.attacks[active_unit.selected_attack]
            # print(f"selected attack{selected_attack}")
            # find the ennemy and attack it

            if attack_position != None:
                attack_animation_playing = True
                attack_animation_position = attack_position
                attack_animation_type = selected_attack
                attack_animation_start_position = active_unit.mapPosition
                animation_start_time = pygame.time.get_ticks()
                for enemy_sprite in players[not (active_player_index)].sprite_managers:
                    if enemy_sprite.mapPosition == attack_position:
                        # print(f"this is the {attack_position}")
                        if hasattr(active_unit, 'perform_special_attack'):
                            print(f"{enemy_sprite.defense}")
                            if not enemy_sprite.defense :
                                active_unit.perform_special_attack(enemy_sprite)
                            else:
                                print(f"{enemy_sprite.name} is in a defensive state and will not take damage. ")
                                enemy_sprite.defense = False
                        else:
                            active_unit.perform_attack(30, enemy_sprite)

                        if not enemy_sprite.is_alive() and not enemy_sprite.marked_for_removal:
                            enemy_sprite.marked_for_removal = True
                            enemy_sprite.removal_time = pygame.time.get_ticks()
                            #players[active_player_index].score += 1

                        #change the player once attacked
                        players[active_player_index].played = 0
                        players[active_player_index].set_active(False)
                        active_player_index = (active_player_index + 1) % len(players)
                        players[active_player_index].set_active(True)
                        last_turn_switch_time = current_time
                
                for monster in self.monsters[:]:
                    if monster.mapPosition == attack_position:
                        damage = 30
                        #attack_animation_playing = True
                        active_unit.perform_attack(damage, monster)
                        if not monster.is_alive()and not monster.marked_for_removal:
                            monster.marked_for_removal = True
                            monster.removal_time = pygame.time.get_ticks()
                            players[active_player_index].score += 500
                            #self.monsters.remove(monster)
                        players[active_player_index].played = 0
                        players[active_player_index].set_active(False)
                        active_player_index = (active_player_index + 1) % len(players)
                        players[active_player_index].set_active(True)
                        last_turn_switch_time = current_time
                        # Increment player score
                        #players[active_player_index].score += 1

            if active_unit.is_slow == True:
                highlighted_positions = self.dungeon_manager.fillDungeon_tiles(unit_position,
                                                                               active_unit.attack_selected,
                                                                               selected_attack,
                                                                               players[active_player_index].played, 
                                                                               self.monsters,
                                                                               active_unit.move_range - 1)
            else:
                highlighted_positions = self.dungeon_manager.fillDungeon_tiles(unit_position,
                                                                               active_unit.attack_selected,
                                                                               selected_attack,
                                                                               players[active_player_index].played,
                                                                               self.monsters,
                                                                               active_unit.move_range)
            # print(highlighted_positions)
            # Updates the units
            for player in players:
                for sprite in player.sprite_managers:
                    sprite.dungeon.fillDungeon_sprites(sprite, sprite == active_unit, self.screen)
            for monster in self.monsters:
                other_monsters = [m for m in self.monsters if m != monster] 
                player_units = [unit for player in players for unit in player.sprite_managers]  
                monster.move_randomly(other_monsters, player_units)

                #print(f"{monster.name} new position: {monster.mapPosition}")        
            for monster in self.monsters:
                self.dungeon_manager.fillDungeon_monsters(monster, self.screen)
                

            if attack_animation_playing:
                if attack_animation_type == "Thunder Strike":
                    self.dungeon_manager.play(attack_animation_type, attack_animation_position)
                elif attack_animation_type == "Fireball":
                    self.dungeon_manager.play(attack_animation_type, attack_animation_position,
                                              start_position=attack_animation_start_position)
                elif attack_animation_type == "Water Splash":
                    self.dungeon_manager.play(attack_animation_type, attack_animation_position)
                elif attack_animation_type == "Tsunami Wave":
                    self.dungeon_manager.play(attack_animation_type, attack_animation_position,
                                              start_position=attack_animation_start_position)
                elif attack_animation_type == "Flame Burst":
                    self.dungeon_manager.play(attack_animation_type, attack_animation_position,
                                              start_position=attack_animation_start_position)
                elif attack_animation_type == "Ice Spike":
                    self.dungeon_manager.play(attack_animation_type, attack_animation_position,
                                              start_position=attack_animation_start_position)
                elif attack_animation_type == "Blizzard":
                    self.dungeon_manager.play(attack_animation_type, attack_animation_position,
                                              start_position=attack_animation_start_position)
                elif attack_animation_type == "Lightning Storm":
                    self.dungeon_manager.play(attack_animation_type, attack_animation_position,
                                              start_position=attack_animation_start_position)
                attack_animation_playing = False

            self.dungeon_manager.tsunami_group.update()
            self.dungeon_manager.tsunami_group.draw(self.screen)
            self.dungeon_manager.fireball_group.update()
            self.dungeon_manager.fireball_group.draw(self.screen)
            self.dungeon_manager.flame_burst_group.update()
            self.dungeon_manager.flame_burst_group.draw(self.screen)
            self.dungeon_manager.ice_spike_group.update()
            self.dungeon_manager.ice_spike_group.draw(self.screen)
            self.dungeon_manager.blizzard_group.update()
            self.dungeon_manager.blizzard_group.draw(self.screen)
            self.dungeon_manager.light_storm_group.update()
            self.dungeon_manager.light_storm_group.draw(self.screen)
            for player in players:
                for sprite in player.sprite_managers[:]:
                    if sprite.marked_for_removal:
                        delay = 1000  
                        if pygame.time.get_ticks() - sprite.removal_time > delay:
                            player.sprite_managers.remove(sprite)
            for monster in self.monsters[:]:
                if monster.marked_for_removal and pygame.time.get_ticks() - monster.removal_time > 500:
                    self.monsters.remove(monster)
            self.display_scores()



            if key_input[K_ESCAPE] or pygame.event.peek(QUIT):
                # update_map(self.file_path, None, '*', (8, 10), distance=2)
                sys.exit()
            #self.dungeon_manager.play("Water Splash", (0,0))
            #self.scree n.blit(self.background, (100, 0))
            #screen = pygame.display.set_mode((800, 600))
            #image = pygame.image.load("data/images/effects/thunder.png")  # Chargez votre image
            #position = (200, 150)  # Position de l'image

            #self.dungeon_manager.display_image_for_one_second([0,0])
            pygame.display.update()


            self.poll_events_with_timeout(185)  # General delay, interruptible

### **4.2 Module des Héros**

#### **Description**
Ce module définit les classes des héros jouables, chacune ayant des caractéristiques et des compétences uniques. Chaque héros possède :
- Un type d’attribut élémentaire (eau, feu, glace, etc.).
- Une portée de déplacement initiale spécifique.
- Des compétences spéciales avec des effets variés.
- Des états possibles (gelé, brûlé, ralenti, etc.).

#### **Fonctionnalités principales**
1. Définir des héros avec des caractéristiques de base différentes.
2. Gérer les compétences spécifiques de chaque héros.
3. Appliquer des effets d’états en fonction des compétences utilisées.

---

#### **1. Caractéristiques de base**
Chaque héros possède des caractéristiques de base, définies lors de l'initialisation :
- **Portée de déplacement (`move_range`)** : Distance maximale que le héros peut parcourir par tour.
- **Attribut élémentaire (`element_type`)** : Détermine les forces et faiblesses en fonction des interactions avec d'autres éléments.
- **Points de vie (`health`)** : Quantité initiale de points de vie.
- **Compétences (`attacks`)** : Liste des attaques que le héros peut utiliser.


In [ ]:
class WaterSprite(spriteManager):
    def __init__(self, dungeon, media, mapPosition, name):
        super().__init__(dungeon, media, mapPosition, move_range=5)
        self.name = name
        self.attacks = ["Water Splash", "Tsunami Wave", "Defense"]
        self.element_type = "Water"
        self.health = 100  # Points de vie de base


#### **2. Portée de déplacement spécifique**
Chaque héros commence avec une **portée de déplacement unique**, qui est déterminée par l’attribut `move_range` :
- Les héros rapides (comme le type "Feu") auront une portée plus grande.
- Les héros lourds ou défensifs (comme le type "Glace") auront une portée plus limitée.


In [ ]:
class FireSprite(spriteManager):
    def __init__(self, dungeon, media, mapPosition, name):
        super().__init__(dungeon, media, mapPosition, move_range=6)  # Portée plus grande
        self.name = name
        self.attacks = ["Fireball", "Flame Burst", "Defense"]
        self.element_type = "Fire"
        self.health = 80

class IceSprite(spriteManager):
    def __init__(self, dungeon, media, mapPosition, name):
        super().__init__(dungeon, media, mapPosition, move_range=3)  # Portée réduite
        self.name = name
        self.attacks = ["Ice Spike", "Blizzard", "Defense"]
        self.element_type = "Ice"
        self.health = 120


#### **3. Compétences spécifiques**
Chaque héros possède une liste de compétences. Les compétences peuvent avoir des effets variés :
- **Effets directs** : Infliger des dégâts à un ennemi.
- **Effets de contrôle** : Appliquer des états comme "Ralentissement" ou "Paralysie".
- **Effets défensifs** : Guérison ou bouclier.

In [ ]:
    def perform_special_attack(self, target):
        print(f"{self.name} unleashes a {self.attacks[1]} at {target.mapPosition}!")

        if self.attacks[self.selected_attack] == "Water Splash":
            base_damage = 20
            base_slow_duration = 1
        elif self.attacks[self.selected_attack] == "Tsunami Wave":
            base_damage = 30
            base_slow_duration = 2

        # Calculate damage and effects
        damage, burn_turn, freeze_turn, paralyze_turn,slow_turn = calculate_damage_and_effect(self, target, base_damage, slow=base_slow_duration)

        # Apply damage or healing
        if damage < 0:
            target.health -= damage
            print(f"{target.name} heals for {-damage} HP!")
        else:
            target.take_damage(damage)

        if slow_turn > 0:
            target.apply_status_effect("Slow", slow_turn)
            print(f"{slow_turn} turns remind")

### **Résumé**
- **Portée de déplacement** : Varie en fonction du type et de la conception du héros.
- **Compétences uniques** : Chaque héros dispose de capacités qui renforcent sa personnalité stratégique.
- **Gestion des états** : Les effets appliqués influencent directement les performances des héros au combat.
- **Extensions** : Ajout futur de nouveaux héros, terrains et mécaniques pour enrichir l’expérience de jeu.

### **4.3 Module de la Carte**
- **Description** : Gère la génération et le rendu de la carte du donjon.
- **Fonctionnalités principales** :
  - Rendu isométrique des tuiles.
  - Gestion des zones non praticables et des mouvements.

In [ ]:
    def fillDungeon_tiles(self, unit_positions, target_position = False,selected_attack=None, played=False, monsters=None, move_range = 2):
        """Render only the tiles of the dungeon."""
        self._stepX = 0 # en pixel
        self._stepY = 150
        self._rewinderStepX = 0 # At the start of each row, self._stepX is reset to self._rewinderStepX ensuring the rows are correctly spaced.
        self._rewinderStepY = 150
        self._row = self._rowTmp #  the current row in the dungeon map to be rendered.
        self._col = self._colTmp # en terme de nombre de colonnes
        self._patch = 30 # to avoid overlap
        # X-coordinate where the current tile should be rendered, 
        self._centeredItemX = self._windowManager.centerItemX(self._dict[self.dungeon[self._col][self._row]]) + 20
                        
        highlight_positions = set()
        for monster in monsters:
            highlight_positions.add(tuple(monster.mapPosition))
        if played == 0:    
            row, col = unit_positions
            if not target_position:
                for dr in range(-move_range, move_range + 1):
                    for dc in range(-move_range, move_range + 1):
                        if abs(dr) + abs(dc) <= move_range:  
                            highlight_positions.add((row + dr, col + dc))
            else:
                highlight_positions.add((row, col))
            self.previous_highlight_positions = highlight_positions

        else: # utiliser le highlighte_positions precedent
            highlight_positions = self.previous_highlight_positions
        
        while True:# boucle pour l'affiche en iso
            current_position = (self._col, self._row)

            # Tile highlight
            if current_position in highlight_positions:
                if not target_position:
                    self._screen.blit(self._dict["R"], (
                    self._centeredItemX + self._stepX, self._stepY - int(self.elevation[self._col][self._row]) * 20))
                else:
                    self._screen.blit(self._dict["R"], (
                    self._centeredItemX + self._stepX, self._stepY - int(self.elevation[self._col][self._row]) * 20))
                    
                    if selected_attack=="Thunder Strike":
                        image = self._media.loadImage(os.path.join('data', 'images', 'effects', 'thunder.png'))
                        #self.play(selected_attack, current_position, image)
                        self._screen.blit(self._dict["C"], (self._centeredItemX + self._stepX + 10,
                                                        self._stepY-40 - int(self.elevation[self._col][self._row]) * 30))

            # Tile rendering
            elif self.dungeon[self._col][self._row] == 'M':
                if int(self.elevation[self._col][self._row]) > 1:
                    for basePatch in range(1, int(self.elevation[self._col][self._row])):
                        self._screen.blit(self._dict["B"], (self._centeredItemX + self._stepX, self._stepY - basePatch * 20)) #   -80 cause it should appear higher
                self._screen.blit(self._dict[self.dungeon[self._col][self._row]], (self._centeredItemX + self._stepX, self._stepY - 80 - int(self.elevation[self._col][self._row]) * 20))
            elif self.dungeon[self._col][self._row] != ' ':
                if int(self.elevation[self._col][self._row]) > 1:
                    for basePatch in range(1, int(self.elevation[self._col][self._row])):
                        self._screen.blit(self._dict["B"], (self._centeredItemX + self._stepX, self._stepY - basePatch * 20))
                self._screen.blit(self._dict[self.dungeon[self._col][self._row]], (self._centeredItemX + self._stepX, self._stepY - int(self.elevation[self._col][self._row]) * 20))

            # Move to the next tile in the row
            self._stepX += 19
            self._stepY += 10
            self._row += 1

            # Check if the end of the row or screen width is reached
            if self._centeredItemX + self._stepX >= 800 - self._patch or self._row >= len(self.dungeon[self._col]):
                self._patch += 19
                self._row = self._rowTmp
                self._col += 1
                self._rewinderStepX -= 19
                self._stepX = self._rewinderStepX
                self._rewinderStepY += 10
                self._stepY = self._rewinderStepY

                if self._centeredItemX + self._stepX <= 0 or self._col >= len(self.dungeon):
                    break
                
        return highlight_positions

### **4.4 Module des Animations**
- **Description** : Gère les animations des attaques et les effets visuels.
- **Fonctionnalités principales** :
  - Animation pour différents types d'attaques.
  - Effets visuels comme des éclairs, des boules de feu, etc.

In [ ]:
def play(self, attack_type, target_position):
    # ...

## **5. Gestion des États et Interactions des Attributs**
### **Description générale**
Dans ce module, les états appliqués aux unités influencent directement leurs performances en combat. Les états actuellement implémentés sont :
1. **Ralentissement (`Slow`)** : Réduit la portée de déplacement d’une unité.
2. **Brûlure (`Burn`)** : Inflige des dégâts continus au début de chaque tour.
3. **Gel (`Frozen`)** : Immobilise complètement l’unité affectée.
4. **Paralysie (`Paralyze`)** : Donne une chance (50%) de perdre un tour d’action.
- **Interactions des Attributs** : Les dommages et les effets sont modifiés en fonction des attributs de l'attaquant et de la cible.
  - Exemple : Une attaque de type "Eau" inflige des dégâts doublés à une unité de type "Feu".
 
### Skills, Damage, and Debuff Effects  ✅

| **Element** | **Skill**         | **Base Damage** | **Debuff Effect**  |
|-------------|-------------------|-----------------|--------------------|
| **Water**   | Water Splash      | 20              | slow (1 turn)      |
|             | Tsunami Wave      | 30              | slow (2 turns)     |
| **Fire**    | Fireball          | 20              | Burn (1 turn)      |
|             | Flame Burst       | 10              | Burn (3 turns)     |
| **Ice**     | Ice Spike         | 20              | Frozen (1 turn)    |
|             | Blizzard          | 10              | Frozen (3 turns)   |
| **Thunder** | Thunder Strike    | 20              | Paralyze (1 turn)  |
|             | Lightning Storm   | 10              | Paralyze (3 turns) |


### Attribute Resistance System ✅
| **Element** | **Strong Against** | **Weak Against** | **Damage Multiplier** | **Special Effect**                                                                                         |
|-------------|--------------------|------------------|-----------------------|------------------------------------------------------------------------------------------------------------|
| **Water**   | Fire               | Thunder          | Fire x2               | Against Ice: Heal (dmg -50, when it's negative will heal the target), no damage <br/>Causes Slow on target |
| **Fire**    | Ice                | Water            | Ice x2.5              | Causes Burn on target                                                                                      |
| **Ice**     | Thunder, Water     | Fire             | Thunder, Water x1.5   | Causes Freeze on target                                                                                    |
| **Thunder** | Water              | Ice              | Water x2              | Causes Paralysis on target                                                                                 |



In [ ]:
ATTRIBUTE_EFFECTS = {
    "Water": {
        "Fire": {"damage_multiplier": 2 , "slow" : 2}, # Water deals double damage to Fire
        "Ice": {"heal": -50, "slow" : 1},             # Water heals Ice instead of damaging it
        "Thunder": {"damage_multiplier": 0.75, "slow" : -2}, # Water deals half damage to Thunder
    },
    "Fire": {
        "Ice": {"damage_multiplier": 2.5, "burn": 3}, # Fire deals 2.5x damage to Ice and adds 3 turns of burn
        "Water": {"damage_multiplier": 0.5, "burn": -2}, # Fire deals half damage to Water and reduces burn by 2 turns
    },
    "Ice": {
        "Water": {"damage_multiplier": 1.5, "freeze": 2}, # Ice deals 1.5x damage to Water and adds 2 turns of freeze
        "Thunder": {"damage_multiplier": 1.5}, # Ice deals 1.5x damage to Thunder
        "Fire": {"damage_multiplier": 0.5}, # Ice deals half damage to Fire
    },
    "Thunder": {
        "Water": {"damage_multiplier": 1.5, "paralyze": 2}, # Thunder deals double damage to Water and adds 2 turns of paralysis
    }
}

### **5.1 Implémentation des Effets de Statut dans le Programme Principal**

#### **Logique principale des effets de statut**
Dans le programme principal `main.py`, l'application et la mise à jour des effets de statut suivent les étapes suivantes :

1. **Déclenchement des effets de statut** :  
   Lorsque les héros ou les monstres exécutent une attaque, certaines compétences appliquent des effets de statut à l'unité cible.

2. **Stockage des effets de statut** :  
   L'unité cible enregistre les effets appliqués ainsi que leur durée dans un dictionnaire appelé `status_effects`.

3. **Mise à jour des effets de statut** :  
   À la fin de chaque tour, la méthode `update_status_effects` est appelée pour réduire la durée de chaque effet et supprimer les effets expirés.

4. **Impact sur le comportement** :  
   Les effets de statut influencent directement le comportement de l'unité lors de la logique de déplacement ou d'attaque, en modifiant ses capacités en fonction des états actuels.


In [ ]:
# in the main loop
    for sprite in players[active_player_index].sprite_managers:
        sprite.update_status_effects()
    
    active_player_index = (active_player_index + 1) % len(players)
    players[active_player_index].set_active(True)
    last_turn_switch_time = current_time  # Update the timestamp

# in the spritemanager

    def apply_status_effect(self, effect_name, duration):
        """
        Applies a status effect to the target.
        :param effect_name: The name of the status effect, e.g., "Frozen", "Burning", "Slowed".
        :param duration: The number of turns the effect will last.
        """
        self.status_effects[effect_name] = duration
        if effect_name == "Burn":
            self.is_burning = True
        elif effect_name == "Frozen":
            self.is_frozen = True
        elif effect_name == "Paralyze":
            self.is_paralyze = True
        elif effect_name == "Slow":
            self.is_slow = True

    def update_status_effects(self):
        """
        Updates status effects each turn, such as reducing their duration and removing expired effects.
        """
        expired_effects = []

        for effect, duration in self.status_effects.items():
            print(f"{self.status_effects[effect]} turns remaining, health is {self.health}")
            self.status_effects[effect] -= 1
            if self.status_effects[effect] <= 0:
                print(f"{effect} is expired")
                expired_effects.append(effect)

        for effect in expired_effects:
            print(expired_effects)
            del self.status_effects[effect]
            if effect == "Frozen":
                self.is_frozen = False
            elif effect == "Burn":
                self.is_burning = False
            elif effect == "Paralyze":
                self.is_paralyze = False
            elif effect == "Slow" :
                self.is_slow = False

        # Persistent damage for burning
        if self.is_burning:
            self.take_damage(10)
            print(f"{self.name} is burned!")

        # Frozen effect: Cannot move
        if self.is_frozen:
            print(f"{self.name} is frozen at {self.mapPosition}!")

        # Paralyze effect: 50% chance to lose action
        if self.is_paralyze:
            if random.random() < 0.5:  # 50% chance
                print(f"{self.name} is paralyzed and cannot take actions this turn!")
            else:
                print(f"{self.name} resists paralysis and can act this turn!")

        # Frozen effect: Cannot move
        if self.is_slow:
            print(f"{self.name} is slow down !")

## **6. Tests et Optimisations**
### **6.1 Méthodologie de Test**
- **Tests fonctionnels** :
  - Vérification du rendu correct de la carte.
  - Validation des états appliqués aux unités.
- **Tests de performance** :
  - Évaluation de la fluidité lors de l'augmentation du nombre d'unités.
  - Analyse de la fréquence d'images pendant les animations.

### **6.2 Stratégies d’Optimisation**
- Minimiser les mises à jour inutiles des états.
- Ajuster la fréquence des animations pour améliorer les performances.

## **7. Implémentation des Effets de Statut dans le Programme Principal**

### **7.1 Logique principale des effets de statut**
Dans le programme principal `main.py`, l'application et la mise à jour des effets de statut suivent les étapes suivantes :

1. **Déclenchement des effets de statut** :  
   Lorsque les héros ou les monstres exécutent une attaque, certaines compétences appliquent des effets de statut à l'unité cible.

2. **Stockage des effets de statut** :  
   L'unité cible enregistre les effets appliqués ainsi que leur durée dans un dictionnaire appelé `status_effects`.

3. **Mise à jour des effets de statut** :  
   À la fin de chaque tour, la méthode `update_status_effects` est appelée pour réduire la durée de chaque effet et supprimer les effets expirés.

4. **Impact sur le comportement** :  
   Les effets de statut influencent directement le comportement de l'unité lors de la logique de déplacement ou d'attaque, en modifiant ses capacités en fonction des états actuels.

---

### **7.2 Problèmes rencontrés lors de l’implémentation des effets de statut**

#### **1. Difficulté à comprendre le code lors de la collaboration**
- **Problème** : Étant un projet collaboratif avec un grand volume de code, il fallait souvent beaucoup de temps pour comprendre les ajouts des autres membres de l’équipe.
- **Solution** : Grâce à une approche structurée :
  - Débogage étape par étape pour vérifier les nouvelles fonctionnalités.
  - Lecture progressive de la logique de la boucle principale (`run_game_loop`) pour comprendre comment les différentes parties du code interagissent.

#### **2. Problème d’héritage entre classes**
- **Problème** : Lors de la mise en œuvre du système de `debuff`, les effets n’étaient pas correctement déclenchés. Cela provenait d’un problème dans la relation d’héritage entre la classe `spriteManager` (classe parent) et ses sous-classes (`FireSprite`, `WaterSprite`, etc.).
- **Solution** : Uniformiser les relations entre les variables des deux classes :
  - Vérification des variables partagées entre la classe parent et ses sous-classes.
  - Centralisation de la gestion des états dans la classe parent `spriteManager`.

#### **3. Expiration incorrecte des effets de statut**
- **Problème** : Les effets de statut, tels que "Ralentissement" ou "Gel", expiraient trop tôt. Le problème venait du fait que les effets étaient mis à jour pour **toutes les unités (alliées et ennemies)** à chaque tour, ce qui réduisait leur durée deux fois plus rapidement que prévu.  
  Exemple : Un effet censé durer 6 tours n’en durait que 3 dans la pratique.
- **Solution** :
  - Après une relecture approfondie de la logique dans `run_game_loop`, le déclenchement des mises à jour des états a été modifié pour ne s’appliquer qu’aux héros alliés.
  - Les mises à jour des états sont maintenant effectuées uniquement **après le tour des alliés**, garantissant une durée correcte.

#### **4. Gestion incorrecte de la portée de déplacement**
- **Problème** : La portée de déplacement des unités ne respectait pas toujours leur attribut `move_range`. Par exemple, une unité avec une portée de déplacement réduite par un effet de "Ralentissement" pouvait encore se déplacer normalement.
- **Solution** :
  - Mise en place d’une gestion dynamique de la portée de déplacement dans `fillDungeon_tiles`, en tenant compte des états de l’unité (par exemple, `is_slow`).

---

### **Résumé**
Grâce à ces solutions, la logique des effets de statut est maintenant robuste et gérée de manière centralisée :
1. **Collaboration optimisée** : Une lecture structurée et un débogage méthodique ont permis de comprendre rapidement les contributions des membres de l’équipe.
2. **Héritage unifié** : La relation entre `spriteManager` et ses sous-classes a été clarifiée, permettant un déclenchement fiable des effets.
3. **Durée correcte des effets** : Les mises à jour des effets sont désormais appliquées uniquement aux alliés, assurant que les effets durent le nombre prévu de tours.
4. **Gestion dynamique** : Les états influencent directement la portée de déplacement et d'autres comportements des unités, améliorant ainsi la stratégie globale du jeu.


## **8. Résumé des Technologies et Logiques Employées**

Ce projet repose sur plusieurs technologies et concepts fondamentaux pour construire un jeu stratégique au tour par tour. Voici un résumé des principales techniques et logiques utilisées à travers les différents fichiers du projet.

---

### **8.1 Technologies utilisées**
1. **Python et Pygame** :
   - **Python** : Langage principal utilisé pour implémenter toute la logique du jeu.
   - **Pygame** : Une bibliothèque essentielle pour gérer les graphismes, les animations, les interactions utilisateur et le rendu des cartes.
     - Gestion des événements (clavier/souris).
     - Chargement des ressources multimédia (images, sons).
     - Rendu des tuiles isométriques et des animations.

2. **Modularité avec Python** :
   - Structure modulaire divisée en plusieurs fichiers tels que `main.py`, `dungeonManager.py`, `spriteManager.py` :
     - **`main.py`** : Boucle principale et gestion des interactions.
     - **`dungeonManager.py`** : Gestion de la carte et des déplacements.
     - **`spriteManager.py`** : Gestion des personnages, états et attaques.

---

### **8.2 Concepts logiques clés**
#### **8.2.1 Gestion des unités**
- **Héritage** :
  - La classe `spriteManager` agit comme une classe parent pour tous les personnages, permettant :
    - Une définition centralisée des attributs communs (par exemple, `mapPosition`, `move_range`, `status_effects`).
    - Une personnalisation des comportements des sous-classes (par exemple, `FireSprite` ou `WaterSprite`).

- **Mise à jour des états** :
  - Les états (`Slow`, `Burn`, etc.) sont gérés via un dictionnaire `status_effects`, et mis à jour chaque tour à l'aide de `update_status_effects`.

#### **8.2.2 Gestion des déplacements**
- **Logique de déplacement** :
  - Les déplacements des unités sont calculés en tenant compte des effets de statut (par exemple, "Ralenti").
  - Utilisation de la méthode `fillDungeon_tiles` pour :
    - Calculer les positions accessibles.
    - Mettre en évidence les tuiles atteignables pour l'unité active.

- **Interactions avec la carte** :
  - La hauteur et les obstacles sur la carte sont gérés via des niveaux d'élévation et des types de tuiles (par exemple, "M" pour montagne, "R" pour chemin).

#### **8.2.3 Gestion des compétences et des combats**
- **Compétences spécifiques** :
  - Chaque héros possède des compétences uniques définies dans `HerosGenerator.py`.
  - Les compétences appliquent des dégâts, des effets de statut ou des bonus de défense.
  
- **Système de dégâts** :
  - Calcul des dégâts basés sur les attributs des unités et les interactions élémentaires (géré dans `Type_Effectiveness.py`).

---

### **8.3 Difficultés techniques surmontées**
1. **Collaboration sur un projet volumineux** :
   - Gestion de la compréhension du code des autres membres via un débogage méthodique.
   - Utilisation d’une structure modulaire pour limiter les conflits de code.

2. **Mise en œuvre des états** :
   - Gestion centralisée des états via `status_effects`.
   - Correction des erreurs de synchronisation des états (par exemple, durée incorrecte des effets).

3. **Optimisation des performances** :
   - Réduction des calculs inutiles dans les déplacements.
   - Gestion efficace des animations avec `anim.py`.

---

### **8.4 Points forts du projet**
1. **Flexibilité** :
   - Une architecture modulaire facilite l'ajout de nouveaux héros, compétences et mécanismes.
   - Une base solide pour ajouter des terrains, des ennemis et d'autres fonctionnalités stratégiques.

2. **Stratégie et gameplay** :
   - Intégration de mécaniques stratégiques via des effets de statut et des interactions élémentaires.
   - Dynamique de déplacement influencée par la carte et les compétences.

3. **Expérience utilisateur immersive** :
   - Rendu graphique fluide grâce à Pygame.
   - Gestion claire des animations et des effets visuels.

---

### **8.5Perspectives d’amélioration**
1. **Équilibrage des héros et compétences** :
   - Ajuster les statistiques et les interactions élémentaires pour garantir un gameplay équilibré.

2. **IA des ennemis** :
   - Ajouter une logique d’intelligence artificielle pour que les ennemis prennent des décisions stratégiques.

3. **Mode multijoueur** :
   - Étendre le jeu pour inclure un mode multijoueur en réseau.

4. **Optimisation des performances** :
   - Réduction des calculs pour de grandes cartes ou des parties avec plusieurs unités.
   - Amélioration de la gestion des ressources (chargement asynchrone des fichiers multimédia).

---

### **Résumé**
Ce projet est une démonstration réussie de l’utilisation de Python et Pygame pour créer un jeu stratégique au tour par tour. Grâce à une structure modulaire et à des mécaniques robustes, il offre une base solide pour des extensions futures tout en proposant une expérience utilisateur engageante et stratégique.
